In [3]:
import pickle
import multirecording_spikeanalysis as spike
import pandas as pd
import numpy as np
from pathlib import Path

In [4]:
# Construct the path in a platform-independent way
path = Path('.') / 'export' / 'finished_curation'

ephys_data = spike.EphysRecordingCollection(str(path))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230628_111202_standard_comp_to_novel_agent_D1_subj_1-1vs1-2and2-2_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230629_111937_standard_comp_to_novel_agent_D2_subj_1-1v1-4and2-1_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Unit 96 is unsorted & has 5811 spikes
Unit 96 will be deleted
Unit 95 is unsorted & has 6458 spikes
Unit 95 will be deleted
20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-4_t3b3L_box1_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_t1b2L_box1_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_mer

In [6]:
timestamp_path = 'rce_per_trial_labeling.xlsx'

# Read the Excel file
timestamp_df = pd.read_excel(timestamp_path, engine='openpyxl')

In [7]:
# Initialize the global dictionary
timestamp_dicts = {}

# Step 1 & 2: Initialize dictionaries for each unique recording_file
unique_files = timestamp_df['recording_file'].unique()
for file in unique_files:
    timestamp_dicts[file] = {cond: [] for cond in ['rewarded', 'win', 'lose', 'omission', 'both_rewarded']}

# Step 3: Fill dictionaries based on conditions
for _, row in timestamp_df.iterrows():
    file = row['recording_file']
    condition = row['condition ']
    # Skip the row if 'condition ' is empty or 'None'
    if pd.isnull(condition) or condition == 'None':
        continue

    time_stamp = row['time_stamp_index'] // 20
    tuple_val = (time_stamp, time_stamp + 10000)
    
    # Determine the specific condition and append tuple_val accordingly
    if condition == 'rewarded':
        timestamp_dicts[file]['rewarded'].append(tuple_val)
    elif condition == 'omission':
        timestamp_dicts[file]['omission'].append(tuple_val)
    elif condition == 'both_rewarded':
        timestamp_dicts[file]['both_rewarded'].append(tuple_val)
    else:
        subj_num = file[file.find('_subj_')+6:file.find('_', file.find('_subj_')+6)].replace('-', '.')
        if condition == subj_num:
            timestamp_dicts[file]['win'].append(tuple_val)
        else:
            timestamp_dicts[file]['lose'].append(tuple_val)

# Step 4: Convert lists to numpy arrays
for file in timestamp_dicts:
    for key in timestamp_dicts[file]:
        timestamp_dicts[file][key] = np.array(timestamp_dicts[file][key])

# Step 5: Remove dictionaries with all empty keys
timestamp_dicts = {file: conds for file, conds in timestamp_dicts.items() if not all(len(conds[key]) == 0 for key in conds)}

# At this point, timestamp_dicts is the global dictionary

In [9]:
for recording in ephys_data.collection.keys():
    # Check if the recording key (without the .rec suffix) is in timestamp_dicts
    recording_key_without_suffix = recording[:-4]  # Remove '.rec' from the end
    if recording_key_without_suffix in timestamp_dicts:
        # Assign the corresponding timestamp_dicts dictionary to event_dict
        ephys_data.collection[recording].event_dict = timestamp_dicts[recording_key_without_suffix]
        
        # Extract the subject from the recording key
        # Find the positions of 'subj_' and '_t' to extract the subject
        start = recording.find('subj_') + 5  # Start index after 'subj_'
        end = recording.find('_t', start)  # End index before '_t'
        subject = recording[start:end].replace('-', '_')  # Replace '-' with '_'
        
        # Assign the extracted subject
        ephys_data.collection[recording].subject = subject

In [10]:
spike_analysis = spike.SpikeAnalysis_MultiRecording(ephys_data)

These recordings are missing event dictionaries:
['20230628_111202_standard_comp_to_novel_agent_D1_subj_1-1vs1-2and2-2_merged.rec', '20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-4_t3b3L_box1_merged.rec', '20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_t1b2L_box1_merged.rec', '20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_t1b3L_box1_merged.rec', '20230629_111937_standard_comp_to_novel_agent_D2_subj_1-4vs1-1and2-2_merged.rec', '20230630_115506_standard_comp_to_novel_agent_D3_subj_1-4vs1-2and2-1_merged_merged.rec']
These recordings are missing subjects: ['20230628_111202_standard_comp_to_novel_agent_D1_subj_1-1vs1-2and2-2_merged.rec', '20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-4_t3b3L_box1_merged.rec', '20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_t1b2L_box1_merged.rec', '20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_t1b3L_box1_merged.rec', '20230629_111937_standard_comp_to_novel_agent_D2_subj_1-4vs1-1and2-2_me

## Starting over with new timestamp labels spreadsheet
### Going from rce_per_trial_labeling to rce_pilot_2_per_video_trial_labels
### Originally used the "2_per_video" spreadsheet but found a different one on Leo's Notion and assumed that was the more up-to-date one

In [11]:
timestamp_path = 'rce_pilot_2_per_video_trial_labels.xlsx'

# Read the Excel file
timestamp_df = pd.read_excel(timestamp_path, engine='openpyxl')

In [12]:
unique_conditions = timestamp_df['condition '].unique().tolist()

# Print the list of unique values
print(unique_conditions)

[nan, 1.4, 1.3, 'rewarded', 1.1, 1.2, 'omission', 'both_rewarded', 2.1, 2.2, 'tie']


In [13]:
# Initialize the global dictionary
timestamp_dicts = {}

# Step 1 & 2: Initialize dictionaries for each unique recording_file
unique_files = timestamp_df['session_dir'].unique()
for file in unique_files:
    timestamp_dicts[file] = {cond: [] for cond in ['rewarded', 'win', 'lose', 'omission', 'both_rewarded', 'tie']}

# Step 3: Fill dictionaries based on conditions
for _, row in timestamp_df.iterrows():
    file = row['session_dir']
    condition = row['condition ']
    # Skip the row if 'condition ' is empty or 'None'
    if pd.isnull(condition) or condition == 'None':
        continue

    time_stamp = row['tone_start_timestamp'] // 20
    tuple_val = (time_stamp, time_stamp + 10000)
    
    # Determine the specific condition and append tuple_val accordingly
    if condition == 'rewarded':
        timestamp_dicts[file]['rewarded'].append(tuple_val)
    elif condition == 'omission':
        timestamp_dicts[file]['omission'].append(tuple_val)
    elif condition == 'both_rewarded':
        timestamp_dicts[file]['both_rewarded'].append(tuple_val)
    elif condition == 'tie':
        timestamp_dicts[file]['tie'].append(tuple_val)
    else:
        subj_num = file[file.find('_subj_')+6:file.find('_', file.find('_subj_')+6)].replace('-', '.')
        if condition == subj_num:
            timestamp_dicts[file]['win'].append(tuple_val)
        else:
            timestamp_dicts[file]['lose'].append(tuple_val)

# Step 4: Convert lists to numpy arrays
for file in timestamp_dicts:
    for key in timestamp_dicts[file]:
        timestamp_dicts[file][key] = np.array(timestamp_dicts[file][key])

# Step 5: Remove dictionaries with all empty keys
timestamp_dicts = {file: conds for file, conds in timestamp_dicts.items() if not all(len(conds[key]) == 0 for key in conds)}

# At this point, timestamp_dicts is the global dictionary

In [14]:
# Construct the path in a platform-independent way
path = Path('.') / 'export' / 'finished_curation'

ephys_data = spike.EphysRecordingCollection(str(path))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230628_111202_standard_comp_to_novel_agent_D1_subj_1-1vs1-2and2-2_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230630_115506_standard_comp_to_novel_agent_D3_subj_1-4vs1-2and2-1_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230629_111937_standard_comp_to_novel_agent_D2_subj_1-1v1-4and2-1_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Unit 96 is unsorted & has 5811 spikes
Unit 96 will be deleted
Unit 95 is unsorted & has 6458 spikes
Unit 95 will be deleted
20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-4_t3b3L_box1_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_t1b2L_box1_

In [15]:
for recording in ephys_data.collection.keys():
    # Check if the recording key (without the .rec suffix) is in timestamp_dicts
    recording_key_without_suffix = recording[:-4]  # Remove '.rec' from the end
    if recording_key_without_suffix in timestamp_dicts:
        # Assign the corresponding timestamp_dicts dictionary to event_dict
        ephys_data.collection[recording].event_dict = timestamp_dicts[recording_key_without_suffix]
        
        # Extract the subject from the recording key
        # Find the positions of 'subj_' and '_t' to extract the subject
        start = recording.find('subj_') + 5  # Start index after 'subj_'
        end = recording.find('_t', start)  # End index before '_t'
        subject = recording[start:end].replace('-', '_')  # Replace '-' with '_'
        
        # Assign the extracted subject
        ephys_data.collection[recording].subject = subject

In [16]:
spike_analysis = spike.SpikeAnalysis_MultiRecording(ephys_data)

These recordings are missing event dictionaries:
['20230628_111202_standard_comp_to_novel_agent_D1_subj_1-1vs1-2and2-2_merged.rec', '20230630_115506_standard_comp_to_novel_agent_D3_subj_1-4vs1-2and2-1_merged.rec', '20230629_111937_standard_comp_to_novel_agent_D2_subj_1-1v1-4and2-1_merged.rec', '20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged.rec', '20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-4_t3b3L_box1_merged.rec', '20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.rec', '20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_t1b2L_box1_merged.rec', '20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged.rec', '20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_t1b2L_box1_merged.rec', '20230628_111202_standard_comp_to_novel_agent_D1_subj_1-2vs1-1and2-1_merged.rec', '20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec', '20230622_110832_standard_comp_to_both_rewarded_D1_sub

In [17]:
timestamp_dicts

{'20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3': {'rewarded': array([[1849109., 1859109.],
         [1909109., 1919109.],
         [1964109., 1974109.],
         [2029109., 2039109.],
         [2084109., 2094109.],
         [2134109., 2144109.],
         [2184109., 2194109.],
         [2289109., 2299109.],
         [2409109., 2419109.],
         [2529109., 2539109.],
         [2639109., 2649109.],
         [2724109., 2734109.],
         [2844109., 2854109.],
         [2904109., 2914109.],
         [2969109., 2979109.],
         [3029109., 3039109.],
         [3104108., 3114108.],
         [3204108., 3214108.],
         [3299108., 3309108.],
         [3349108., 3359108.],
         [1849109., 1859109.],
         [1909109., 1919109.],
         [1964109., 1974109.],
         [2029109., 2039109.],
         [2084109., 2094109.],
         [2134109., 2144109.],
         [2184109., 2194109.],
         [2289109., 2299109.],
         [2409109., 2419109.],
         [2529109., 2539